In [ ]:
#import libraries

import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import re

In [ ]:
#dataset

RAW = "/content/drive/MyDrive/Medicare_Part_D_Prescribers_by_Provider_and_Drug_2023.csv"
CHUNKSIZE = 250_000

USECOLS = ["Prscrbr_NPI","Prscrbr_Type","Prscrbr_State_Abrvtn","Gnrc_Name","Brnd_Name","Tot_Clms","Tot_Benes"]

#OUT_INTERMEDIATE = "Users/ericabrooks/Documents/BioTech_ML/projects/Care-Gap HCP Segmentation/data/intermediate/diabetes_rows.parquet"
#OUT_FINAL        = "Users/ericabrooks/Documents/BioTech_ML/projects/Care-Gap HCP Segmentation/data/out/hcp_caregap_features.csv"

OUT_INTERMEDIATE = "data/intermediate/diabetes_rows.parquet"
OUT_FINAL        = "data/out/hcp_caregap_features.csv"

Path("data/raw").mkdir(parents=True, exist_ok=True)
Path("data/intermediate").mkdir(parents=True, exist_ok=True)
Path("data/out").mkdir(parents=True, exist_ok=True)


In [ ]:
#normalization

def norm_name(s: str) -> str:
    """lowercase, remove punctuation, strip dosage/salt suffixes"""
    if not isinstance(s, str): return ""
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s/+\-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    tails = [" hcl"," hydrochloride"," sodium"," phosphate"," sulfate",
             " er"," xr"," sr"," dr"," cr"," extended release"," mr"," odt"]
    for t in tails:
        if s.endswith(t):
            s = s[:-len(t)]
    return s

def split_combo(norm_generic_or_brand: str) -> list[str]:
    """split combination drugs into components; tolerate separators"""
    if not norm_generic_or_brand: return []
    s = norm_generic_or_brand.replace("+", "/").replace(" and ", "/").replace(" with ", "/")
    parts = re.split(r"[\/\-]", s)
    parts = [p.strip() for p in parts if p.strip()]
    return parts

In [ ]:
#vocab and combos

FIRST_LINE = {"metformin","glucophage","fortamet","glumetza","riomet"}

SULFONYLUREA = {"glipizide","glyburide","glimepiride","chlorpropamide","tolazamide","tolbutamide"}
MEGLITINIDE  = {"repaglinide","nateglinide"}
ALPHA_GLU    = {"acarbose","miglitol"}
TZD          = {"pioglitazone","rosiglitazone","actos","avandia"}

# DPP-4 (you can treat as innovative or keep separate; here we include in innovative)
DPP4 = {"sitagliptin","saxagliptin","linagliptin","alogliptin","januvia","onglyza","tradjenta","nesina"}

# GLP-1
GLP1 = {"semaglutide","liraglutide","dulaglutide","exenatide","lixisenatide","ozempic","rybelsus","victoza","trulicity","bydureon","byetta","adlyxin"}

# GIP/GLP-1 dual
DUAL_GLP1_GIP = {"tirzepatide","mounjaro"}  # add brands as needed

# SGLT2
SGLT2 = {"empagliflozin","dapagliflozin","canagliflozin","ertugliflozin","jardiance","farxiga","invokana","steglatro"}

# Insulins (token list; covers brands & molecule stems)
INSULIN_TOKENS = {"insulin","degludec","glargine","detemir","aspart","lispro","glulisine","nph","regular",
                  "lantus","toujeo","basaglar","tresiba","levemir","novolog","humalog","apidra","fiasp","humulin","novolin"}

def token_to_category(token: str) -> str:
    """map a single molecule/brand token to a high-level category"""
    if token in FIRST_LINE or token.startswith("metformin"):              return "first_line"
    if token in GLP1 or token in DUAL_GLP1_GIP:                           return "innovative"
    if token in SGLT2:                                                    return "innovative"
    if token in DPP4:                                                     return "dpp4"
    if token in SULFONYLUREA or token in MEGLITINIDE or token in ALPHA_GLU: return "legacy"
    if token in TZD:                                                      return "tzd"
    if token in INSULIN_TOKENS or token.startswith("insulin"):            return "insulin"
    return "other"

def row_categories(gnrc: str, brnd: str) -> list[str]:
    """derive categories from generic/brand, handling combos and unknowns"""
    name = norm_name(gnrc) if (isinstance(gnrc, str) and gnrc.strip()) else norm_name(brnd)
    parts = split_combo(name)
    if not parts and name:
        parts = name.split()  # fallback simple tokenization
    cats = [token_to_category(p) for p in parts]
    cats = [c for c in cats if c != "other"]
    return cats

def apportion(total: float, k: int):
    """split a count evenly across k components"""
    if k <= 0: return []
    share = float(total) / k
    return [share] * k



In [ ]:
# ============= PASS 1: STREAM + MAP + PRE-AGG =============
chunked_frames = []
reader = pd.read_csv(RAW, usecols=USECOLS, chunksize=CHUNKSIZE, dtype=str, low_memory=False)

for i, chunk in enumerate(reader, 1):
    # numeric coercion
    for c in ["Tot_Clms","Tot_Benes"]:
        chunk[c] = pd.to_numeric(chunk[c], errors="coerce").fillna(0)

    # build compact rows with apportioning for combos
    rows = []
    for _, r in chunk.iterrows():
        cats = row_categories(r["Gnrc_Name"], r["Brnd_Name"])
        if not cats:
            continue
        k = len(cats)
        clm_shares  = apportion(r["Tot_Clms"],  k)
        bene_shares = apportion(r["Tot_Benes"], k)
        for cat, cs, bs in zip(cats, clm_shares, bene_shares):
            rows.append((
                r["Prscrbr_NPI"], r["Prscrbr_Type"], r["Prscrbr_State_Abrvtn"],
                cat, cs, bs
            ))

    if not rows:
        continue

    chunk_out = pd.DataFrame(
        rows,
        columns=["npi","specialty_description","nppes_provider_state",
                 "drug_category","total_claim_count","bene_count"]
    )

    # pre-aggregate within chunk (reduces memory before concatenation)
    g = (chunk_out.groupby(
            ["npi","specialty_description","nppes_provider_state","drug_category"],
            dropna=False
         )
         .agg(total_claim_count=("total_claim_count","sum"),
              bene_count=("bene_count","sum"))
         .reset_index())
    chunked_frames.append(g)

    if i % 10 == 0:
        print(f"[pass1] processed {i} chunks…")

# combine and aggregate across all chunks
if chunked_frames:
    agg = pd.concat(chunked_frames, ignore_index=True)
    agg = (agg.groupby(
                ["npi","specialty_description","nppes_provider_state","drug_category"],
                dropna=False
           )
           .agg(total_claim_count=("total_claim_count","sum"),
                bene_count=("bene_count","sum"))
           .reset_index())
else:
    agg = pd.DataFrame(columns=[
        "npi","specialty_description","nppes_provider_state","drug_category",
        "total_claim_count","bene_count"
    ])

# optional: save the long-form diabetes rows for QC
try:
    agg.to_parquet(OUT_INTERMEDIATE, index=False)
except Exception:
    pass

[pass1] processed 10 chunks…
[pass1] processed 20 chunks…
[pass1] processed 30 chunks…
[pass1] processed 40 chunks…
[pass1] processed 50 chunks…
[pass1] processed 60 chunks…
[pass1] processed 70 chunks…
[pass1] processed 80 chunks…
[pass1] processed 90 chunks…
[pass1] processed 100 chunks…


In [ ]:
# ============= PASS 2: PIVOT TO PHYSICIAN (WIDE) =============
hcp = agg.pivot_table(
    index=["npi","specialty_description","nppes_provider_state"],
    columns="drug_category",
    values="total_claim_count",
    aggfunc="sum",
    fill_value=0
).reset_index()
hcp.columns.name = None

# patient volume (beneficiaries) across categories
bene = agg.pivot_table(
    index=["npi","specialty_description","nppes_provider_state"],
    columns="drug_category",
    values="bene_count",
    aggfunc="sum",
    fill_value=0
).reset_index()
bene.columns.name = None
bene["patient_volume"] = bene.drop(columns=["npi","specialty_description","nppes_provider_state"]).sum(axis=1)

hcp = hcp.merge(
    bene[["npi","specialty_description","nppes_provider_state","patient_volume"]],
    on=["npi","specialty_description","nppes_provider_state"],
    how="left"
)

# ensure all expected category columns exist
for col in ["first_line","innovative", "dpp4","legacy","insulin","tzd"]:
    if col not in hcp.columns: hcp[col] = 0.0

hcp["total_rx"] = hcp[["first_line","innovative","dpp4","legacy","insulin","tzd"]].sum(axis=1)

In [ ]:
# ============= FEATURES & SCORES =============
def safe_div(num, den):
    den = den.replace(0, pd.NA)
    return num / den

hcp["pct_first_line"]   = safe_div(hcp["first_line"],   hcp["total_rx"])
hcp["pct_innovative"]   = safe_div(hcp["innovative"],   hcp["total_rx"])
hcp["pct_dpp4"]         = safe_div(hcp["dpp4"],         hcp["total_rx"])
hcp["pct_legacy"]       = safe_div(hcp["legacy"],       hcp["total_rx"])
hcp["pct_insulin"]      = safe_div(hcp["insulin"],      hcp["total_rx"])
hcp["pct_tzd"]          = safe_div(hcp["tzd"],          hcp["total_rx"])

eps = 1e-6
hcp["care_gap_score"]   = (hcp["pct_legacy"].fillna(0) + hcp["pct_insulin"].fillna(0)) / (hcp["pct_first_line"].fillna(0) + eps)
hcp["innovation_score"] = hcp["pct_innovative"].fillna(0)

# flags (tune to your distribution later)
hcp["flag_high_gap"]        = (hcp["pct_first_line"] < 0.20) & (hcp["pct_legacy"] > 0.30)
hcp["flag_innovator"]       = (hcp["innovation_score"] > 0.40)
hcp["flag_guideline_aligned"]= (hcp["pct_first_line"] > 0.40) & (hcp["pct_legacy"] < 0.15)

hcp["flag_dpp4_heavy"]      = (hcp["pct_dpp4"] > 0.25)

FINAL_COLS = [
    "npi","specialty_description","nppes_provider_state",
    "total_rx","patient_volume",
    "first_line","innovative","dpp4","legacy","insulin","tzd",
    "pct_first_line","pct_innovative","pct_dpp4","pct_legacy","pct_insulin","pct_tzd",
    "care_gap_score","innovation_score",
    "flag_high_gap","flag_innovator","flag_guideline_aligned", "flag_dpp4_heavy"
]
hcp = hcp[FINAL_COLS]

# ============= SAVE =============
hcp.to_csv(OUT_FINAL, index=False)
print(f"[done] wrote {OUT_FINAL} with {len(hcp):,} HCP rows")

[done] wrote data/out/hcp_caregap_features.csv with 291,983 HCP rows
